In [1]:
import requests
import pandas as pd
import json
import numpy as np

In [2]:
datelist_2016 = pd.date_range('2016-01-01', periods=365)
datelist_2015 = pd.date_range('2015-01-01', periods=365)
datelist_2014 = pd.date_range('2014-01-01', periods=365)
datelist_2013 = pd.date_range('2013-01-01', periods=365)
datelist_2012 = pd.date_range('2012-01-01', periods=365)
datelist_2011 = pd.date_range('2011-01-01', periods=365)
datelist_2010 = pd.date_range('2010-01-01', periods=365)
datelist_2016 = [str(i.date()) for i in datelist_2016]
datelist_2015 = [str(i.date()) for i in datelist_2015]
datelist_2014 = [str(i.date()) for i in datelist_2014]
datelist_2013 = [str(i.date()) for i in datelist_2013]
datelist_2012 = [str(i.date()) for i in datelist_2012]
datelist_2011 = [str(i.date()) for i in datelist_2011]
datelist_2010 = [str(i.date()) for i in datelist_2010]


In [ ]:
def get_complaints(dates):
    complaints = []
    for date in dates:
        r = requests.get(f'https://data.cityofnewyork.us/resource/fhrw-4uyv.json?created_date={date}')
        data = r.json()
        complaints.extend(data)
    return complaints
        

In [ ]:

# complaints_2015 = get_complaints(datelist_2015)
# complaints_2014 = get_complaints(datelist_2014)
# complaints_2013 = get_complaints(datelist_2013)
# complaints_2012 = get_complaints(datelist_2012)
# complaints_2011 = get_complaints(datelist_2011)
# complaints_2010 = get_complaints(datelist_2010)

In [3]:
service_requests = pd.read_csv('311_Service_Requests_from_2010_to_Present.csv')

/Users/briansrebrenik/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
service_requests.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,32310363,12/31/2015 11:59:45 PM,01/01/2016 12:55:15 AM,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10034.0,71 VERMILYEA AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.865682,-73.923501,"(40.86568153633767, -73.92350095571744)"
1,32309934,12/31/2015 11:59:44 PM,01/01/2016 01:26:57 AM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,11105.0,27-07 23 AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.775945,-73.915094,"(40.775945312321085, -73.91509393898605)"
2,32309159,12/31/2015 11:59:29 PM,01/01/2016 04:51:03 AM,NYPD,New York City Police Department,Blocked Driveway,No Access,Street/Sidewalk,10458.0,2897 VALENTINE AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.870325,-73.888525,"(40.870324522111424, -73.88852464418646)"
3,32305098,12/31/2015 11:57:46 PM,01/01/2016 07:43:13 AM,NYPD,New York City Police Department,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,10461.0,2940 BAISLEY AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.835994,-73.828379,"(40.83599404683083, -73.82837939584206)"
4,32306529,12/31/2015 11:56:58 PM,01/01/2016 03:24:42 AM,NYPD,New York City Police Department,Illegal Parking,Blocked Sidewalk,Street/Sidewalk,11373.0,87-14 57 ROAD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.733060,-73.874170,"(40.733059618956815, -73.87416975810375)"


In [8]:
service_requests.columns

Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Park Facility Name', 'Park Borough',
       'School Name', 'School Number', 'School Region', 'School Code',
       'School Phone Number', 'School Address', 'School City', 'School State',
       'School Zip', 'School Not Found', 'School or Citywide Complaint',
       'Vehicle Type', 'Taxi Company Borough', 'Taxi Pick Up Location',
       'Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp',
       'Bridge Highway Segment', 'Garage Lot Name', 'Ferry 